Connected to agent (Python 3.10.14)

In [1]:
import json
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv("../../.env")

True

In [2]:
client = OpenAI()
client

In [3]:
# 定义检索鲜花库存的函数
def get_flower_inventory(city):
    """获取指定城市的鲜花库存"""
    if "北京" in city:
        return json.dumps({"city": "北京", "inventory": "玫瑰: 100, 郁金香: 150"})
    elif "上海" in city:
        return json.dumps({"city": "上海", "inventory": "百合: 80, 康乃馨: 120"})
    elif "深圳" in city:
        return json.dumps({"city": "深圳", "inventory": "向日葵: 200, 玉兰: 90"})
    else:
        return json.dumps({"city": city, "inventory": "未知"})

# 工具（也就是函数）的元数据
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_flower_inventory",
            "description": "获取指定城市的鲜花库存",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "城市名称，例如：北京、上海或深圳"
                    }
                },
                "required": ["city"]
            }
        }
    }
]

# 第一次对话的Message
messages = [{"role": "user", "content": "北京、上海和深圳的鲜花库存是多少？"}]
print("message:", messages)

message: [{'role': 'user', 'content': '北京、上海和深圳的鲜花库存是多少？'}]


In [4]:
# 第一次对话的返回结果
first_response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)
print("first_response:", first_response)

first_response: ChatCompletion(id='chatcmpl-9gCgFzR9YvIgYgi6pPlDm5s5bTARl', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_IavKkzEHwy3ORDikFwh2CELu', function=Function(arguments='{"city": "北京"}', name='get_flower_inventory'), type='function'), ChatCompletionMessageToolCall(id='call_Y0NIsjAkxMRmpqFMkwqI7Jcy', function=Function(arguments='{"city": "上海"}', name='get_flower_inventory'), type='function'), ChatCompletionMessageToolCall(id='call_Sv0JEiJueQnBtNh0thOmGnUE', function=Function(arguments='{"city": "深圳"}', name='get_flower_inventory'), type='function')]))], created=1719845547, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=67, prompt_tokens=95, total_tokens=162))


In [5]:
response_message = first_response.choices[0].message
tool_calls = response_message.tool_calls

# 如果返回结果要求用Function Call，就调用函数，并把函数的查询结果附加到消息中
if tool_calls:
    messages.append(response_message)
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        function_response = get_flower_inventory(
            city = function_args.get("city")
        )
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )
print("message:", messages)

message: [{'role': 'user', 'content': '北京、上海和深圳的鲜花库存是多少？'}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_IavKkzEHwy3ORDikFwh2CELu', function=Function(arguments='{"city": "北京"}', name='get_flower_inventory'), type='function'), ChatCompletionMessageToolCall(id='call_Y0NIsjAkxMRmpqFMkwqI7Jcy', function=Function(arguments='{"city": "上海"}', name='get_flower_inventory'), type='function'), ChatCompletionMessageToolCall(id='call_Sv0JEiJueQnBtNh0thOmGnUE', function=Function(arguments='{"city": "深圳"}', name='get_flower_inventory'), type='function')]), {'tool_call_id': 'call_IavKkzEHwy3ORDikFwh2CELu', 'role': 'tool', 'name': 'get_flower_inventory', 'content': '{"city": "\\u5317\\u4eac", "inventory": "\\u73ab\\u7470: 100, \\u90c1\\u91d1\\u9999: 150"}'}, {'tool_call_id': 'call_Y0NIsjAkxMRmpqFMkwqI7Jcy', 'role': 'tool', 'name': 'get_flower_inventory', 'content': '{"city": "\\u4e0a\\u6d77", "inventory": "\\u767e\\u5408: 8

In [6]:
# 用有了库存查询结果的Message再来一次对话
second_response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages
    )
print("second_response:", second_response)

second_response: ChatCompletion(id='chatcmpl-9gCgHeGTnDZBYP6k2opMS4MQ4qJau', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='北京的鲜花库存为：玫瑰 100支、牡丹香 150支。\n\n上海的鲜花库存为：百合 80支、康乃馨 120支。\n\n深圳的鲜花库存为：向日葵 200支、玉兰 90支。', role='assistant', function_call=None, tool_calls=None))], created=1719845549, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=90, prompt_tokens=235, total_tokens=325))
